# Positional Encoding

Como ya vimos, en el lenguaje es muy importante la posición de las palabras en la frase, ya que `Juan quiere a María` no significa lo mismo que `María quiere a Juan`.

Pero como el transformer veía la secuencia entera y creaba las relaciones mediante los mecanismos de atencíon, no tenía manera de saber la posición de cada token en la frase.

Por lo que mediante la adición de un número pequeño (entre -1 y 1) podíamos dar la información de posición en la frase a esos mecanismos de atención

Por tanto, en el decoder es necesario también un `Positional Encoding`

<div style="text-align:center;">
  <img src="Imagenes/transformer_architecture_model_decoder_positional_embedding.png" alt="Decoder positional encoding" style="width:425px;height:626px;">
</div>

Esta información se da medainte las ecuaciones

<div style="text-align:center;">
  <img src="Imagenes/positional_encoding.png" alt="positional encoding">
</div>

Que modifican el valor de los embeddings entre -1 y 1, lo cual no cambiaba mucho la posición del embedding dentro del espacio vectorial

<div style="text-align:center;">
  <img src="Imagenes/word_embedding_3_dimmension.png" alt="word embedding 3 dimmension" style="width:662px;height:467px;">
</div>

Porque como se puede ver en la figura anterior, los embeddings tienen valores muy grandes, por lo que esta pequeña modificación, no los perturbará mucho

## Implementación

Aquí también reusamos la clase que creamos en el encoder

In [1]:
import torch
import torch.nn as nn
import math

class PositionalEncoding(nn.Module):
    def __init__(self, max_sequence_len, embedding_model_dim):
        """
        Args:
            seq_len: length of input sequence
            embed_model_dim: demension of embedding
        """
        super().__init__()
        self.embedding_dim = embedding_model_dim

        # create constant 'positional_encoding' matrix with values dependant on pos and i
        positional_encoding = torch.zeros(max_sequence_len, self.embedding_dim)
        for pos in range(max_sequence_len):
            for i in range(0, self.embedding_dim, 2):
                positional_encoding[pos, i]     = math.sin(pos / (10000 ** ((2 *     i) / self.embedding_dim)))
                positional_encoding[pos, i + 1] = math.cos(pos / (10000 ** ((2 * (i+1)) / self.embedding_dim)))
        positional_encoding = positional_encoding.unsqueeze(0)
        self.register_buffer('positional_encoding', positional_encoding)
        # Esta última línea es equivalente a hacer self.positional_encoding = positional_encoding
        # sin embargo al hacerlo así PyTorch no lo considerará parte del estado del modelo.
        # Hay varias implicaciones de esto:
        #  * Movimiento de dispositivos: Cuando se mueve el modelo a la GPU con model.to(device), 
        #    PyTorch automáticamente moverá todos los parámetros y buffers registrados al dispositivo especificado.
        #    Sin embargo, no moverá los tensores que no sean parámetros o buffers registrados. Por lo tanto, 
        #    si se asigna positional_encoding a self.positional_encoding directamente, habría que moverlo 
        #    manualmente a la GPU.
        #  * Serialización: Cuando se guarda el modelo con torch.save(model.state_dict(), PATH), PyTorch guardará 
        #    todos los parámetros y buffers registrados del modelo. Pero no guardará tensores que no son parámetros 
        #    o buffers registrados. Por lo tanto, si se asigna positional_encoding a self.positional_encoding 
        #    directamente, no se guardará cuando se guarde el estado del modelo.
        #  * Modo de evaluación: Algunos métodos, como model.eval(), cambian el comportamiento de ciertas capas del 
        #    modelo (como Dropout o BatchNorm) dependiendo de si el modelo está en modo de entrenamiento o evaluación.
        #    Para que estas capas funcionen correctamente, PyTorch necesita conocer su estado actual, que se almacena
        #    en sus parámetros y buffers registrados. Si positional_encoding no está registrado como un buffer, 
        #    entonces no será afectado por el cambio de modo.
        # En resumen, si no se usa register_buffer para positional_encoding, se tendrían que manejar estas cosas 
        # manualmente, lo cual podría ser propenso a errores.
        # La principal ventaja de usar register_buffer() es que PyTorch se encargará de estas cosas por nosotros.


    def forward(self, x):
        """
        Args:
            x: input vector
        Returns:
            x: output
        """
        # make embeddings relatively larger
        x = x * math.sqrt(self.embedding_dim)
        
        # add encoding matrix to embedding (x)
        sequence_len = x.size(1)
        # x = x + torch.autograd.Variable(self.positional_encoding[:,:sequence_len], requires_grad=False)
        x = x + self.positional_encoding[:,:sequence_len]
        return x

Vamos a probarlo

In [5]:
embedding_dim = 512
max_sequence_len = 100
positional_encoding = PositionalEncoding(max_sequence_len, embedding_dim)
x = torch.randn(1, max_sequence_len, embedding_dim)
output = positional_encoding(x)
print(f"input shape: {x.shape}, output shape: {output.shape}")

input shape: torch.Size([1, 100, 512]), output shape: torch.Size([1, 100, 512])


Como vemos recibe una matriz con un tamaño dado y la devuelve del mismo tamaño, solo que la ha modificado un poco